# Première analyse du dataset

## Quelques informations sur les données

L’objectif est de développer **un modèle performant** pour prédire la **variable Output** en fonction des **autres variables pertinentes**. La plupart des variables ont un nom décrivant l’information contenue.

 

**Quelques informations :**

* Les variables finissant en _M : variables calculées au mois M
* Les variables finissant en _MY : variables calculées au mois M-Y
* Les variables finissant en _XM : variables calculées sur les mois M-X au mois M 



### Librairies pour l'analyse

In [56]:
import pandas as pd
import numpy as np 
import seaborn as sns
import math

# Chargement des données

In [19]:
df = pd.read_csv('Dataset_test_ILB.csv',encoding="latin1",sep=';',decimal=',')
# Encodage latin1 car le fichier n'est pas encodé en UTF8, selection de "," en séparateur décimal

## Lecture du fichier et reformattage de certaines composantes si nécessaire

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3015 entries, 0 to 3014
Data columns (total 27 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   ID                                    3015 non-null   int64  
 1   Nombres_adultes                       3005 non-null   float64
 2   Nombre_personnes_foyer                3006 non-null   float64
 3   Age_2eme_personne                     3013 non-null   float64
 4   Note_credit                           3010 non-null   float64
 5   Encours_credit                        2993 non-null   object 
 6   ID_PP                                 3008 non-null   object 
 7   Encours_credit_habitat                3011 non-null   float64
 8   Montant_flux_crediteur_M1             3011 non-null   float64
 9   Montant_obligations                   3011 non-null   float64
 10  Montant_operation_crediteur_DAV_M1    3002 non-null   float64
 11  Montant_operation

In [24]:
df.head()

,ID,Nombres_adultes,Nombre_personnes_foyer,Age_2eme_personne,Note_credit,Encours_credit,ID_PP,Encours_credit_habitat,Montant_flux_crediteur_M1,Montant_obligations,...,Nombre_retrait_carte_M1,"Nombre_retrait_carte_M1,1",Quantité_obligations,Nombre_connexion_appli_6M,Nombre_de_connexion_appli_semaine_6M,Nombre_connexion_site_6M,Duree_connexion_moyenne_6M,Montant_flux_crediteur_M,Montant_pret_banque,Output
0,2911,2.057430,2.070880,91.042848,0.177448,"4715,49819",732f2de9240c404122f8c97b952c4cb8be8ede7f,2772.907651,9005.374276,5897.737602,...,0.160537,0.094456,262736.76140,46.683372,41.184480,48.057303,28.811906,5757.583993,124370.2918,FAUX
1,1398,2.060715,2.193722,90.715413,-0.062708,"6584,320922",1cc77d640ec2c383bef7099ff331d2ae962fd712,7094.588072,5047.824214,55655.446030,...,-0.172851,-0.170338,157962.64670,6.806368,9.210575,12.027955,-8.664351,2825.730247,130015.4600,FAUX
2,1793,1.930472,1.946847,90.588554,-0.094438,"-7792,256696",34d7809e534d6e7d83e1439cd4032201386fe9dd,-7613.883523,-692.852208,2277.193995,...,-0.037880,-0.086625,-26034.01144,3.442761,4.988904,7.072354,-21.135796,1679.601033,130904.7126,FAUX
3,1193,1.963631,1.916677,89.300012,-0.286729,"6118,791869",1539c6f04d9403928ef4e5b662490bd4787ad71d,5874.214001,1408.588664,8280.325647,...,0.994902,0.001148,336491.25330,-6.298359,-7.873857,-9.758806,-6.232009,4059.963478,123729.3778,FAUX
4,1342,2.009754,2.019892,87.706161,0.284325,"-4266,843489",27524f055875f70d102006ee6dcb360e982c4a69,-4459.202674,-889.296675,59977.521970,...,-0.027045,0.080714,-10631.26532,3.056277,4.414500,5.862660,-28.520162,1719.696619,126770.7544,FAUX


In [106]:
df.shape

(3015, 27)

La majorité des colonnes qui contiennt des float ont bien été converties, sauf **Encours_credit**
> **Encours_credit**: Argent qui a été prêté mais qui n'a pas encore été remboursé.

## [Encours_credit]  - Reformattage

In [120]:
df.Encours_credit[14:34]

14    PAS DENCOURS
15    -4648,965858
16     9654,794858
17    -14351,44645
18    -1636,208879
19    -9442,465683
20    -8047,237148
21    -2059,018592
22     3981,916098
23     -2517,34467
24    -6878,210721
25     182,6623636
26    -5287,245058
27    -10360,55858
28      11300,4433
29    -3482,976268
30     10768,15036
31    -18316,31167
32     2641,386869
33             NaN
Name: Encours_credit, dtype: object

On comprend mieux pourquoi la conversion n'a pas pu se faire: il faut prendre en compte les personnes qui n'ont pas d'encours credit (PAS DENCOURS), ou pour lesquels on ne connaît pas l'information (NaN)

#### Formattage d'encours_credit

Pour que l'encours credit garde son sens, je propose de séparer cette colonne en deux colonnes : 

| Encours_credit_bool             | Encours_credit_value |
| ------------------------------- | -------------------- |
| True                            | -1151                |
| False (Remplace 'PAS DENCOURS') | NaN                  |



In [76]:
"""Création de deux array encours_credit_bool & encours_credit_value """
def create_encours_credit_bool_and_value(df):
    encours_credit = df['Encours_credit'].tolist()
    """Fonction qui construit les deux array évoqués ci-dessus"""
    encours_credit_bool = [True]*len(encours_credit)
    encours_credit_value =[np.nan]*len(encours_credit)
    for (i,encours) in enumerate(encours_credit): 
        """Parcourt la liste encours_credit : encours est un str pour le moment"""
        if encours =="PAS DENCOURS":
            encours_credit_bool[i] = False
        else : 
            """Il y a bien un encours, il va falloir convertir le str en float : attention sep ',' """
            try :
                """Il se peut que encours soit un NaN c'est pourquoi on utilise try & except"""
                encours_float = '.'.join(encours.split(',')) # Utilisation de split pour la conversion
                encours_float = float(encours_float)
                encours_credit_value[i] = encours_float
            except : 
                encours_credit_bool[i] = np.nan
                
    """On peut renvoyer les array"""
    return(encours_credit_bool,encours_credit_value)

In [75]:
encours_credit_bool,encours_credit_values = create_encours_credit_bool_and_value(df)

On vérifie si la structure est conforme à ce que nous avions envisagé

In [86]:
encours = pd.DataFrame(zip(encours_credit_bool,encours_credit_values),columns=['Encours_credit_bool','Encours_credit_value'])

In [87]:
encours.iloc[13:34,:]

,Encours_credit_bool,Encours_credit_value
13,True,5286.493638
14,False,NaN
15,True,-4648.965858
16,True,9654.794858
17,True,-14351.446450
18,True,-1636.208879
19,True,-9442.465683
20,True,-8047.237148
21,True,-2059.018592
22,True,3981.916098


In [105]:
encours.shape

(3015, 2)

#### Modification d'encours credit dans le dataframe

In [108]:
df1 = df.copy()

On va insérer notre notre dataframe encours à la place de 'Encours_credit'

In [109]:
df1.iloc[:2,:5]

,ID,Nombres_adultes,Nombre_personnes_foyer,Age_2eme_personne,Note_credit
0,2911,2.057430,2.070880,91.042848,0.177448
1,1398,2.060715,2.193722,90.715413,-0.062708


In [110]:
encours.head(2)

,Encours_credit_bool,Encours_credit_value
0,True,4715.498190
1,True,6584.320922


In [111]:
df1.iloc[:2,6:]

,ID_PP,Encours_credit_habitat,Montant_flux_crediteur_M1,Montant_obligations,Montant_operation_crediteur_DAV_M1,Montant_operation_crediteur_DAV_M,Montant_operation_debiteur_DAV_M1,Montant_operation_debiteur_DAV_M,Montant_retrait_carte_M,Montant_retrait_carte_M1,...,Nombre_retrait_carte_M1,"Nombre_retrait_carte_M1,1",Quantité_obligations,Nombre_connexion_appli_6M,Nombre_de_connexion_appli_semaine_6M,Nombre_connexion_site_6M,Duree_connexion_moyenne_6M,Montant_flux_crediteur_M,Montant_pret_banque,Output
0,732f2de9240c404122f8c97b952c4cb8be8ede7f,2772.907651,9005.374276,5897.737602,4498.397638,7930.203493,4997.109487,9488.923224,31.338746,25.008051,...,0.160537,0.094456,262736.7614,46.683372,41.184480,48.057303,28.811906,5757.583993,124370.2918,FAUX
1,1cc77d640ec2c383bef7099ff331d2ae962fd712,7094.588072,5047.824214,55655.446030,1890.630239,-1655.250533,1669.661800,1494.971252,-0.191790,-3.399387,...,-0.172851,-0.170338,157962.6467,6.806368,9.210575,12.027955,-8.664351,2825.730247,130015.4600,FAUX


In [114]:
def concat_df_encours(df,encours):
    """On sépare df en deux parties, encours_credit étant la 6 colonne de df"""
    part1 = df.iloc[:,:5]
    part2 = df.iloc[:,6:]
    df1 = pd.concat([part1,encours],axis=1)
    df1 = pd.concat([df1,part2],axis=1)
    return(df1)

In [116]:
df1 = concat_df_encours(df,encours)

In [117]:
df1.head()

,ID,Nombres_adultes,Nombre_personnes_foyer,Age_2eme_personne,Note_credit,Encours_credit_bool,Encours_credit_value,ID_PP,Encours_credit_habitat,Montant_flux_crediteur_M1,...,Nombre_retrait_carte_M1,"Nombre_retrait_carte_M1,1",Quantité_obligations,Nombre_connexion_appli_6M,Nombre_de_connexion_appli_semaine_6M,Nombre_connexion_site_6M,Duree_connexion_moyenne_6M,Montant_flux_crediteur_M,Montant_pret_banque,Output
0,2911,2.057430,2.070880,91.042848,0.177448,True,4715.498190,732f2de9240c404122f8c97b952c4cb8be8ede7f,2772.907651,9005.374276,...,0.160537,0.094456,262736.76140,46.683372,41.184480,48.057303,28.811906,5757.583993,124370.2918,FAUX
1,1398,2.060715,2.193722,90.715413,-0.062708,True,6584.320922,1cc77d640ec2c383bef7099ff331d2ae962fd712,7094.588072,5047.824214,...,-0.172851,-0.170338,157962.64670,6.806368,9.210575,12.027955,-8.664351,2825.730247,130015.4600,FAUX
2,1793,1.930472,1.946847,90.588554,-0.094438,True,-7792.256696,34d7809e534d6e7d83e1439cd4032201386fe9dd,-7613.883523,-692.852208,...,-0.037880,-0.086625,-26034.01144,3.442761,4.988904,7.072354,-21.135796,1679.601033,130904.7126,FAUX
3,1193,1.963631,1.916677,89.300012,-0.286729,True,6118.791869,1539c6f04d9403928ef4e5b662490bd4787ad71d,5874.214001,1408.588664,...,0.994902,0.001148,336491.25330,-6.298359,-7.873857,-9.758806,-6.232009,4059.963478,123729.3778,FAUX
4,1342,2.009754,2.019892,87.706161,0.284325,True,-4266.843489,27524f055875f70d102006ee6dcb360e982c4a69,-4459.202674,-889.296675,...,-0.027045,0.080714,-10631.26532,3.056277,4.414500,5.862660,-28.520162,1719.696619,126770.7544,FAUX


In [118]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3015 entries, 0 to 3014
Data columns (total 28 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   ID                                    3015 non-null   int64  
 1   Nombres_adultes                       3005 non-null   float64
 2   Nombre_personnes_foyer                3006 non-null   float64
 3   Age_2eme_personne                     3013 non-null   float64
 4   Note_credit                           3010 non-null   float64
 5   Encours_credit_bool                   2993 non-null   object 
 6   Encours_credit_value                  2970 non-null   float64
 7   ID_PP                                 3008 non-null   object 
 8   Encours_credit_habitat                3011 non-null   float64
 9   Montant_flux_crediteur_M1             3011 non-null   float64
 10  Montant_obligations                   3011 non-null   float64
 11  Montant_operation

**[Modification de encours credit]**
> A l'issue de cette première partie nous avons pu retravailler le dataframe et notamment la colonne "Encours_credit" afin de simplifier l'analyse

## Analyse des données

Maintenant que nous avons chargés le dataset et reformaté les données qui se devaient d'être reformatées, nous allons pouvoir regarder : 
* Les features, valeurs, répartitions
* Output, répartitions

In [125]:
df = df1.copy() # On sauvegarde sur df les changements réalisés précedemment

In [128]:
df.columns

Index(['ID', 'Nombres_adultes', 'Nombre_personnes_foyer', 'Age_2eme_personne',
       'Note_credit', 'Encours_credit_bool', 'Encours_credit_value', 'ID_PP',
       'Encours_credit_habitat', 'Montant_flux_crediteur_M1',
       'Montant_obligations', 'Montant_operation_crediteur_DAV_M1',
       'Montant_operation_crediteur_DAV_M',
       'Montant_operation_debiteur_DAV_M1', 'Montant_operation_debiteur_DAV_M',
       'Montant_retrait_carte_M', 'Montant_retrait_carte_M1', 'CSP',
       'Nombre_retrait_carte_M1', 'Nombre_retrait_carte_M1,1',
       'Quantité_obligations', 'Nombre_connexion_appli_6M',
       'Nombre_de_connexion_appli_semaine_6M', 'Nombre_connexion_site_6M',
       'Duree_connexion_moyenne_6M', 'Montant_flux_crediteur_M',
       'Montant_pret_banque', 'Output'],
      dtype='object')

In [131]:
features_cols = df.columns[1:-1] # Sélection des colonnes qui contiennent les features (de la 2e à l'avant-dernière col)

### Analyse des features

In [132]:
features = df[features_cols]

In [137]:
features.columns

Index(['Nombres_adultes', 'Nombre_personnes_foyer', 'Age_2eme_personne',
       'Note_credit', 'Encours_credit_bool', 'Encours_credit_value', 'ID_PP',
       'Encours_credit_habitat', 'Montant_flux_crediteur_M1',
       'Montant_obligations', 'Montant_operation_crediteur_DAV_M1',
       'Montant_operation_crediteur_DAV_M',
       'Montant_operation_debiteur_DAV_M1', 'Montant_operation_debiteur_DAV_M',
       'Montant_retrait_carte_M', 'Montant_retrait_carte_M1', 'CSP',
       'Nombre_retrait_carte_M1', 'Nombre_retrait_carte_M1,1',
       'Quantité_obligations', 'Nombre_connexion_appli_6M',
       'Nombre_de_connexion_appli_semaine_6M', 'Nombre_connexion_site_6M',
       'Duree_connexion_moyenne_6M', 'Montant_flux_crediteur_M',
       'Montant_pret_banque'],
      dtype='object')

In [138]:
### Il faut enlever ID_PP qui n'est pas une features
features = features.drop(columns='ID_PP')

In [139]:
features

,Nombres_adultes,Nombre_personnes_foyer,Age_2eme_personne,Note_credit,Encours_credit_bool,Encours_credit_value,Encours_credit_habitat,Montant_flux_crediteur_M1,Montant_obligations,Montant_operation_crediteur_DAV_M1,...,CSP,Nombre_retrait_carte_M1,"Nombre_retrait_carte_M1,1",Quantité_obligations,Nombre_connexion_appli_6M,Nombre_de_connexion_appli_semaine_6M,Nombre_connexion_site_6M,Duree_connexion_moyenne_6M,Montant_flux_crediteur_M,Montant_pret_banque
0,2.057430,2.070880,91.042848,0.177448,True,4715.498190,2772.907651,9005.374276,5897.737602,4498.397638,...,CSP+,0.160537,0.094456,262736.761400,46.683372,41.184480,48.057303,28.811906,5757.583993,124370.2918
1,2.060715,2.193722,90.715413,-0.062708,True,6584.320922,7094.588072,5047.824214,55655.446030,1890.630239,...,CSP-,-0.172851,-0.170338,157962.646700,6.806368,9.210575,12.027955,-8.664351,2825.730247,130015.4600
2,1.930472,1.946847,90.588554,-0.094438,True,-7792.256696,-7613.883523,-692.852208,2277.193995,4433.692768,...,CSP+,-0.037880,-0.086625,-26034.011440,3.442761,4.988904,7.072354,-21.135796,1679.601033,130904.7126
3,1.963631,1.916677,89.300012,-0.286729,True,6118.791869,5874.214001,1408.588664,8280.325647,3513.923067,...,CSP-,0.994902,0.001148,336491.253300,-6.298359,-7.873857,-9.758806,-6.232009,4059.963478,123729.3778
4,2.009754,2.019892,87.706161,0.284325,True,-4266.843489,-4459.202674,-889.296675,59977.521970,5713.900960,...,CSP+,-0.027045,0.080714,-10631.265320,3.056277,4.414500,5.862660,-28.520162,1719.696619,126770.7544
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3010,0.878986,0.820440,-6.848754,6.606900,True,46221.671070,47115.820070,758.719360,-8391.547055,-793.005389,...,CSP-,0.064015,-0.056925,-106444.157700,-2.581037,-0.638313,-1.028824,-2.837844,551.265893,127823.1016
3011,0.912940,1.010965,-6.973332,6.866550,True,98357.789320,98926.300610,491.383808,6830.338232,-803.998408,...,CSP+,0.075142,0.131996,317236.284700,6.074446,8.513984,10.661164,-18.050579,536.016885,128499.9149
3012,0.869316,0.873146,-10.140412,7.301042,True,82070.149300,81384.823830,-527.577382,471.694560,2790.978232,...,CSP+,0.912718,1.923661,-2617.422193,3.859618,4.194136,5.803531,86.268629,1068.225793,130983.6234
3013,1.002031,1.052272,NaN,-0.349542,True,-10916.291840,-10437.097280,4606.670590,14999.062190,NaN,...,CSP-,-0.043945,-0.014223,331778.676700,3.250432,6.662248,8.955402,-18.184747,1821.002855,131433.2856


#### Corrélations entre features

In [140]:
features.corr()

,Nombres_adultes,Nombre_personnes_foyer,Age_2eme_personne,Note_credit,Encours_credit_value,Encours_credit_habitat,Montant_flux_crediteur_M1,Montant_obligations,Montant_operation_crediteur_DAV_M1,Montant_operation_crediteur_DAV_M,...,Montant_retrait_carte_M1,Nombre_retrait_carte_M1,"Nombre_retrait_carte_M1,1",Quantité_obligations,Nombre_connexion_appli_6M,Nombre_de_connexion_appli_semaine_6M,Nombre_connexion_site_6M,Duree_connexion_moyenne_6M,Montant_flux_crediteur_M,Montant_pret_banque
Nombres_adultes,1.000000,0.098466,0.002919,0.019444,0.029628,0.028885,0.014248,0.033160,0.014294,0.010862,...,0.003933,-0.005540,-0.014507,-0.004507,0.002225,0.002284,0.001369,0.015531,0.010593,0.016287
Nombre_personnes_foyer,0.098466,1.000000,-0.007328,0.181072,0.262180,0.255145,0.046759,-0.114754,0.101532,0.078258,...,0.046181,0.016469,0.035351,-0.028180,0.069666,0.081170,0.071683,0.104936,0.102839,0.135540
Age_2eme_personne,0.002919,-0.007328,1.000000,-0.031807,-0.020675,-0.020063,-0.008253,0.055883,-0.003136,-0.005832,...,0.017860,0.005160,0.003618,0.007822,0.010698,0.003511,0.002672,-0.010746,-0.002676,0.000104
Note_credit,0.019444,0.181072,-0.031807,1.000000,0.579427,0.560637,0.030100,-0.199724,0.089110,0.107423,...,-0.091995,-0.081144,-0.065752,-0.059746,0.058460,0.066848,0.065102,0.149016,0.066847,-0.027083
Encours_credit_value,0.029628,0.262180,-0.020675,0.579427,1.000000,0.998138,0.101685,-0.144385,0.187738,0.150310,...,-0.110063,-0.101642,-0.085494,-0.040975,0.016034,0.031155,0.027703,0.175748,0.065089,0.430152
Encours_credit_habitat,0.028885,0.255145,-0.020063,0.560637,0.998138,1.000000,0.100283,-0.140766,0.180761,0.147899,...,-0.111558,-0.100582,-0.087366,-0.039918,0.013298,0.028346,0.025540,0.175861,0.055765,0.442342
Montant_flux_crediteur_M1,0.014248,0.046759,-0.008253,0.030100,0.101685,0.100283,1.000000,-0.013607,0.067405,0.152328,...,-0.029262,-0.031120,-0.027696,0.003796,0.083247,0.078123,0.079474,0.055675,0.036849,0.107695
Montant_obligations,0.033160,-0.114754,0.055883,-0.199724,-0.144385,-0.140766,-0.013607,1.000000,-0.012001,0.026005,...,0.009071,0.002898,0.001503,0.673195,-0.027024,-0.028536,-0.023300,-0.065601,-0.012019,-0.012633
Montant_operation_crediteur_DAV_M1,0.014294,0.101532,-0.003136,0.089110,0.187738,0.180761,0.067405,-0.012001,1.000000,0.238838,...,-0.037728,-0.031801,-0.029361,0.009352,0.069351,0.065431,0.058467,0.062445,0.599463,0.148354
Montant_operation_crediteur_DAV_M,0.010862,0.078258,-0.005832,0.107423,0.150310,0.147899,0.152328,0.026005,0.238838,1.000000,...,-0.036883,-0.036627,-0.031245,0.060740,0.043868,0.060124,0.057344,0.036467,0.249502,0.061574


### Des "catégories dans les features" :

> On remarque que l'on peut classer les features selon leurs appartenances aux catégories suivantes

| Catégories | Informations globales <br> (Informations concernant la personne, age, nbr adultes, montant du crédit, crédit restant) | Opérations bancaires <br> (Ensemble des opérations effectuées sur le mois) | Extra financiers <br> (Informations sur le comportement de la personne, suivi de ses opérations bancaire) |
| ---------- | ------------------------------------------------------------ | ------------------------------------------------------------ | ------------------------------------------------------------ |
| Features   | Nombre_adultes \| Nombre_personnes_foyer <br> Age_2e_personne \| Note_credit \| Montant_pret_banque \| Encours | Montant_operation_crediteur/debiteur <br> Nombre de retraits <br> Montant_flux_créditeur | Nombre_connexion_appli <br> Nombre_connexion_site <br> Durée de connexion moyenne |

**En regroupant selon ses catégories, l'analyse de corrélation sera plus pertinente**

In [127]:
df.describe()

,ID,Nombres_adultes,Nombre_personnes_foyer,Age_2eme_personne,Note_credit,Encours_credit_value,Encours_credit_habitat,Montant_flux_crediteur_M1,Montant_obligations,Montant_operation_crediteur_DAV_M1,...,Montant_retrait_carte_M1,Nombre_retrait_carte_M1,"Nombre_retrait_carte_M1,1",Quantité_obligations,Nombre_connexion_appli_6M,Nombre_de_connexion_appli_semaine_6M,Nombre_connexion_site_6M,Duree_connexion_moyenne_6M,Montant_flux_crediteur_M,Montant_pret_banque
count,3015.000000,3005.000000,3006.000000,3.013000e+03,3010.000000,2970.000000,3011.000000,3011.000000,3.011000e+03,3002.000000,...,3008.000000,3011.000000,3010.000000,3.009000e+03,2994.000000,2994.000000,2996.000000,3006.000000,3009.000000,3008.00000
mean,1505.492206,1.651430,2.139211,2.247984e+03,4.087619,67733.356217,66183.088854,3430.032454,1.057915e+04,8039.496539,...,25.181340,0.141723,0.150699,1.941102e+05,21.343223,26.634469,33.320717,102.902545,4069.769709,129798.31574
std,869.645684,3.073339,1.130802,7.714272e+04,3.901568,97274.363255,97054.712799,12601.220620,3.639671e+04,34477.188935,...,127.211080,0.815417,0.976184,2.198428e+06,42.840104,57.345026,74.547646,151.341643,17200.093454,50784.99592
min,0.000000,-45.154000,0.628834,-1.014041e+01,-1.243272,-35268.964050,-35446.980780,-12073.545780,-1.111274e+04,-7999.359456,...,-61.640257,-0.276593,-0.287192,-5.665139e+05,-12.976680,-15.128918,-19.493009,-43.124721,-5509.113441,-12165.56655
25%,752.500000,1.023015,1.099215,1.193945e+00,0.060387,-49.319006,-1058.635777,-355.231937,1.663585e+02,8.222620,...,-8.283941,-0.045147,-0.050422,-4.443938e+04,0.940546,1.122001,1.411483,7.331611,494.447401,124794.80720
50%,1506.000000,1.958080,1.979002,3.295269e+01,6.071823,15300.678100,12956.683190,2173.796979,4.705268e+03,2600.692478,...,1.666568,0.010977,0.006696,7.685730e+04,6.721483,8.460031,10.834154,57.338650,1963.882364,129489.98350
75%,2259.500000,2.011098,2.634577,4.842538e+01,6.646759,122237.835025,121627.982600,5120.094253,9.941821e+03,5664.498512,...,12.740980,0.066336,0.073643,2.001135e+05,23.635741,29.717792,36.966530,150.928176,3801.005236,134389.96840
max,2999.000000,157.000000,8.029462,3.651695e+06,14.147356,779946.571100,779588.699300,397248.044000,1.003670e+06,462366.112900,...,2406.320620,17.061262,23.098115,1.001995e+08,786.034686,1271.847271,1731.175163,2085.949319,367964.823600,722428.89020
